### Import libraries

In [1]:
import random
import string

### Define functions
* Any random word in certain length

In [2]:
def generate_word(length):
    result = ''
    
    x = ''.join(random.sample(string.ascii_letters + string.digits, k=length))
    # ascii_letters: 문자, digits: 숫자
    
    return x

* Population in given size and length

In [3]:
def generate_population(size, min_len, max_len):
    population = []
    
    for i in range(size):
        
        length = random.randint(min_len, max_len)
        # random.randint(a,b): a,b사이에서 랜덤으로 정수 추출
        
        population.append(generate_word(length))
        
    return population

* Fitness of the candidates

In [4]:
def fitness(password, test_word):
    score = 0

    if len(password) != len(test_word):
        return score
    
    else:
        len_score = 0.5
        score += len_score
        # 길이가 맞으면 0.5점 부여
        
        for i in range(len(password)):
            if password[i] == test_word[i]:
                score += 1
                # 문자와 위치가 맞으면 1점 부여

    return score / (len(password) + len_score) * 100

* Evaluate the performance of candidates

In [5]:
def compute_performace(population, password):
    performance_list = []
    
    for chromosome in population:
        score = fitness(password, chromosome)

        if score > 0:
            pred_len = len(chromosome)
        performance_list.append([chromosome, score])

    population_sorted = sorted(performance_list, key=lambda x: x[1], reverse=True)
    # sorted(a,b): a에서 b 기준으로 오름차순 (여기서는 score 기준으로 내림차순)
    return population_sorted, pred_len

* Selecting

In [6]:
def select_survivors(population_sorted, best_sample, lucky_sample, password_len):
    next_generation = []

    for i in range(best_sample):
        if population_sorted[i][1] > 0:
            next_generation.append(population_sorted[i][0])
            # population_sorted에서 가장 점수가 높은 chromosome을 best_sample만큼 가져옴

    lucky_survivors = random.sample(population_sorted, k=lucky_sample)
    # random.sample(a,b): a에서 b개를 랜덤으로 추출
    
    for i in lucky_survivors:
        next_generation.append(i[0])
        # 운좋게 살아남은 survivors 추가
        
    while len(next_generation) < best_sample + lucky_sample:
        next_generation.append(generate_word(length=password_len))
        # 다음 세대의 수가 부족하다면 새로운 개체 추가

    random.shuffle(next_generation)
    return next_generation

* Uniform 

In [7]:
def create_child(chromosome1, chromosome2):
    child = ''
    min_len_chro = min(len(chromosome1), len(chromosome2))
    
    for i in range(min_len_chro):
        
        if (int(100 * random.random()) < 50):
            child += chromosome1[i]
        else:
            child += chromosome2[i]
        # 50%의 확률로 부모 중 하나를 따라감
            
    return child

In [8]:
def create_children(parents, num_child):
    next_population = []
    
    for i in range(int(len(parents)/2)):
        # 부모 수/2 = 커플 쌍
        
        for j in range(num_child):
            next_population.append(create_child(parents[i], parents[len(parents) - 1 - i]))
            # 각 커플쌍마다 num_child 만큼의 자녀 생성
            
    return next_population

* Mutation

def mutate_word(word):
    idx = int(random.random() * len(word))
    
    if (idx == 0):
        word = random.choice(string.ascii_letters + string.digits) + word[1:]
        
    else:
        word = word[:idx] + random.choice(string.ascii_letters + string.digits) + word[idx+1:]
        # 임의로 한자리를 랜덤한 문자/숫자로 변경
        
    return word

In [9]:
def mutate_word(word):
    for i in range(len(word)):
        if word[i] != password[i]:
            word[i] = random.choice(string.ascii_letters + string.digits)
            
    
    return word    

In [10]:
def mutate_population(population, chance_of_mutation):
    for i in range(len(population)):
        if random.random() * 100 < chance_of_mutation:
            population[i] = mutate_word(population[i])
    return population

#### Test

In [11]:
password = '1HanDong1'
min_len = 2
max_len = 10
n_generation = 300
population = 100
best_sample = 20
lucky_sample = 20
num_child = 5
chance_of_mutation = 10

pop = generate_population(size=population, min_len=min_len, max_len=max_len)

for g in range(n_generation):
    pop_sorted, pred_len = compute_performace(population=pop, password=password)

    if int(pop_sorted[0][1]) == 100:
        print('SUCCESS! The password is %s' % (pop_sorted[0][0]))
        break
    
    survivors = select_survivors(population_sorted=pop_sorted, best_sample=best_sample, lucky_sample=lucky_sample, password_len=pred_len)
    print("Password length must be",pred_len)
    
    children = create_children(parents=survivors, num_child=num_child)

    new_generation = mutate_population(population=children, chance_of_mutation=10)
    
    pop = new_generation
    
    print('===== %sth Generation =====' % (g + 1))
    print(pop_sorted[0])

Password length must be 9


TypeError: 'str' object does not support item assignment